In [7]:
import csv
import matplotlib.pyplot as plt
import nltk
import pandas
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import plot_confusion_matrix, f1_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn import tree
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_validate
# import nltk
# nltk.download('stopwords')

pandas.set_option('display.max_rows', 1000)

# loading the dataset
dataset = pandas.read_csv("training_set.csv",sep=',',usecols=['tweet','label'])

#print(type(dataset))
print(f"Number of tweets: {len(dataset)}")
display(dataset)

Number of tweets: 1056


,tweet,label
0,vaccini: il successo della brexit premetterã ...,1
1,aifa autorizza il vaccino astrazeneca ma ne ra...,1
2,vaccini mentre l'eu cerca (tra l'altro senza s...,0
3,si andrebbe presumibilmente ben oltre l'anno. ...,2
4,"oggi 30 gennaio , ho ricevuto la seconda dose ...",1
...,...,...
1051,the_huge_ giusepperisso vivianabillo se una pe...,2
1052,covid-19: la cina approva il suo primo vaccino...,2
1053,ponytaele ladyonorato che senso avrebbe questa...,2
1054,coolo_fun lilianaarmato chetempochefa lei si v...,1


In [8]:
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

italian_stemmer = SnowballStemmer('italian')
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([italian_stemmer.stem(w) for w in analyzer(doc)])

In [9]:
# SIMPLE PIPELINE WITHOUT ANY TEST ON ngram_range AND CONSIDERING ONLY ACCURACY AS METRIC
# SE GLI ALTRI TEST VANNO BENE QUESTO SI PUO' ANCHE ELIMINARE

text_clf = Pipeline([
    ('vect', StemmedCountVectorizer(min_df=2, analyzer="word", stop_words = set(stopwords.words('italian')),ngram_range = (1,1))),
    ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True)),
    ('clf', MultinomialNB()),
])
folds =  10
#calculating accuracies in cross-validation
scores = cross_val_score(text_clf, dataset.tweet, dataset.label, cv=folds)
("Accuracy MultinomialNB : %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
#print(scores)

'Accuracy MultinomialNB : 0.50 (+/- 0.15)'

In [10]:
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score, average = 'weighted', zero_division = 0),
           'recall' : make_scorer(recall_score, average = 'macro'), 
           'f1_score' : make_scorer(f1_score, average = 'macro')
          }

def print_metrics(scores):
    print(f'accuracy on test set: {scores["test_accuracy"].mean():.3} +/- {scores["test_accuracy"].std()*2:.3}')
    print(f'precision on test set: {scores["test_precision"].mean():.3} +/- {scores["test_precision"].std()*2:.3}')
    print(f'recall on test set: {scores["test_recall"].mean():.3} +/- {scores["test_recall"].std()*2:.3}')
    print(f'f1-score on test set: {scores["test_f1_score"].mean():.3} +/- {scores["test_f1_score"].std()*2:.3}')

In [11]:
max_ngram = 2

print('NAIVE BAYES CLASSIFIER')
for i in range(max_ngram):
    for j in range(max_ngram):
        if j>= i :
            text_clf = Pipeline([
                ('vect', StemmedCountVectorizer(min_df=2, analyzer="word", stop_words = set(stopwords.words('italian')),ngram_range = (i+1,j+1))),
                ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True)),
                ('clf', MultinomialNB()),
            ])
            scores = cross_validate(text_clf, dataset.tweet, dataset.label, cv=folds, scoring = scoring)
            print(f"------- ngram = {(i+1,j+1)} -------")
            print_metrics(scores)

NAIVE BAYES CLASSIFIER
------- ngram = (1, 1) -------
accuracy on test set: 0.498 +/- 0.154
precision on test set: 0.499 +/- 0.149
recall on test set: 0.498 +/- 0.153
f1-score on test set: 0.496 +/- 0.153
------- ngram = (1, 2) -------
accuracy on test set: 0.508 +/- 0.169
precision on test set: 0.51 +/- 0.168
recall on test set: 0.508 +/- 0.169
f1-score on test set: 0.505 +/- 0.169
------- ngram = (2, 2) -------
accuracy on test set: 0.447 +/- 0.12
precision on test set: 0.449 +/- 0.12
recall on test set: 0.447 +/- 0.119
f1-score on test set: 0.444 +/- 0.116


In [12]:
print('DECISION TREE CLASSIFIER')
for i in range(max_ngram):
    for j in range(max_ngram):
        if j>= i :
            text_clf = Pipeline([
                ('vect', StemmedCountVectorizer(min_df=2, analyzer="word", stop_words = set(stopwords.words('italian')),ngram_range = (i+1,j+1))),
                ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True)),
                ('clf', tree.DecisionTreeClassifier()),
            ])
            scores = cross_validate(text_clf, dataset.tweet, dataset.label, cv=folds, scoring = scoring)
            print(f"------- ngram = {(i+1,j+1)} -------")
            print_metrics(scores)

DECISION TREE CLASSIFIER
------- ngram = (1, 1) -------
accuracy on test set: 0.392 +/- 0.103
precision on test set: 0.393 +/- 0.103
recall on test set: 0.392 +/- 0.104
f1-score on test set: 0.39 +/- 0.0998
------- ngram = (1, 2) -------
accuracy on test set: 0.39 +/- 0.0881
precision on test set: 0.391 +/- 0.0919
recall on test set: 0.39 +/- 0.0884
f1-score on test set: 0.387 +/- 0.0905
------- ngram = (2, 2) -------
accuracy on test set: 0.376 +/- 0.0691
precision on test set: 0.381 +/- 0.0799
recall on test set: 0.376 +/- 0.0689
f1-score on test set: 0.367 +/- 0.0687


In [13]:
print('SVM CLASSIFIER')
for i in range(max_ngram):
    for j in range(max_ngram):
        if j>= i :
            text_clf = Pipeline([
                ('vect', StemmedCountVectorizer(min_df=2, analyzer="word", stop_words = set(stopwords.words('italian')),ngram_range = (i+1,j+1))),
                ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True)),
                ('clf', svm.LinearSVC()),
            ])
            scores = cross_validate(text_clf, dataset.tweet, dataset.label, cv=folds, scoring=scoring)
            print(f"------- ngram = {(i+1,j+1)} -------")
            print_metrics(scores)
            #print(scores)

SVM CLASSIFIER
------- ngram = (1, 1) -------
accuracy on test set: 0.473 +/- 0.141
precision on test set: 0.471 +/- 0.145
recall on test set: 0.473 +/- 0.141
f1-score on test set: 0.47 +/- 0.143
------- ngram = (1, 2) -------
accuracy on test set: 0.469 +/- 0.15
precision on test set: 0.467 +/- 0.153
recall on test set: 0.469 +/- 0.15
f1-score on test set: 0.467 +/- 0.151
------- ngram = (2, 2) -------
accuracy on test set: 0.429 +/- 0.128
precision on test set: 0.432 +/- 0.128
recall on test set: 0.429 +/- 0.128
f1-score on test set: 0.428 +/- 0.125


In [ ]:
"""
from sklearn.metrics import precision_recall_fscore_support
def overall_average_score(actual,prediction):
    precision, recall, f1_score, support = precision_recall_fscore_support(
        actual, prediction, average=None)
    print(type(f1_score))
    return f1_score.mean()

custom_scorer = make_scorer(overall_average_score)


text_clf = Pipeline([
    ('vect', StemmedCountVectorizer(min_df=2, analyzer="word", stop_words = set(stopwords.words('italian')),ngram_range = (1,1))),
    ('tfidf', TfidfTransformer(smooth_idf=True,use_idf=True)),
    ('clf', MultinomialNB()),
])
scores = cross_val_score(text_clf, dataset.tweet, dataset.label, cv=folds, scoring = custom_scorer)
print(scores) """